# Interrupción Legal del Embarazo

## Parameters

In [ ]:
BASE_DIR = '/Users/efraflores/Desktop/EF/Contests/Datathon_CDMX/data' # Mac
# BASE_DIR = r'D:\Users\efras\Documents\EF\Contests\Datathon_CDMX\data' # Windows
RESOURCE_ID = '932b56bf-c5ec-4815-9814-370d58754002' ##### API SÓLO TRAE INFO 2016
FILE_NAME = 'interrupcion_embarazo'
N_CLUSTERS = 10

## Import

In [ ]:
from mariachis.models import InterrupcionEmbarazo
ile = InterrupcionEmbarazo(BASE_DIR, FILE_NAME)
print(ile)

In [ ]:
##### API SÓLO TRAE INFO 2016
ile.full_import(api=True, api_export=False, resource_id=RESOURCE_ID)['ano'].value_counts(1)

In [ ]:
df = ile.full_import(api=False)
df.sample()

## Diccionario de variables

### Limpiar

In [ ]:
# Diccionario para aplicar limpieza por tipo de variable
clean_dict = {}

# Variables que o son 0 o > 0
clean_dict['vars_numbin'] = ['naborto', 'npartos', 'ncesarea', 'nile']

# Numéricas que serán partidas en rangos
clean_dict['vars_num'] = {
    'edad': [9,11,15,18,21,25,29,35],
    'menarca': [7,11,12,13,14,15],
    'fsexual': [7,11,15,18,21],
    'sememb': [3,6,9,12,20],
    'nhijos': [0,1,2],
    'gesta': [0,1,2,3,4],
    'c_num': [0,1,2],
    'p_semgest': [0,5,6,7,8,9],
    'p_diasgesta': [0,1,2,3,4,5]
}

# Obtener la primer palabra
clean_dict['vars_first_word'] = ['anticonceptivo',  'panticoncep']

# Binarias de sí o no
clean_dict['vars_yes_no'] = ['autoref', 'consejeria', 'p_consent', 's_complica', 'c_dolor', 'tanalgesico', 'cconsejo', 'resultado_ile']

# Limpiar y mantener para clustering posterior
clean_dict['vars_clean_keep'] = ['motiles', 'desc_servicio', 'anticonceptivo', 'panticoncep', 'procile_simplificada', 'procile_completa']

# Sólo limpiar
clean_dict['vars_just_clean'] = ['entidad','alc_o_municipio']

# Agrupar categorías
clean_dict['vars_cat'] = {
    'edocivil_descripcion':{
        'SOLTERA':'SOLTERX', 
        'UNION LIBRE':'UNION_LIBRE', 
        'CASADA':'CASADX',
        'DIVORCIADA':'SEPARADX',
        'SEPARADA':'SEPARADX',
        'VIUDA':'SEPARADX',
    },
    'ocupacion':{
        'EMPLEADA':'EMPLEADX',
        'ESTUDIANTE':'ESTUDIANTE',
        'TRABAJADORA DEL HOGAR NO REMUNERADA':'TRAB_HOGAR_NO_REMUNERADX',
        'DESEMPLEADA':'DESEMPLEADX',
    },
    'desc_derechohab':{
        'NINGUNO':'NINGUNO',
        'IMSS':'ALGUNO',
        'SEGURO POPULAR':'ALGUNO',
        'OTRA':'ALGUNO',
        'ISSSTE':'ALGUNO',
    },
    'nivel_edu':{
        'LICENCIATURA':'LICENCIATURA',
        'MAESTRIA':'POSGRADO',
        'DOCTORADO':'POSGRADO',
        'PREPARATORIA':'PREPARATORIA',
        'SECUNDARIA':'SECUNDARIA',
        'PRIMARIA':'PRIMARIA',
        'SIN ACCESO A LA EDUCACION FORMAL':'SIN_ACCESO',
    },
    'parentesco':{
        'PAREJA':'PAREJA',
        'EX-PAREJA':'PAREJA',
        'FAMILIAR MUJER':'FAMILIAR',
        'FAMILIAR HOMBRE':'FAMILIAR',
        'FAMILIAR SIN ESPECIFICAR':'FAMILIAR',
        'TUTOR O RESPONSABLE LEGAL':'FAMILIAR',
        'PERSONA CERCANA':'AMIGX',
        'PERSONA CERCANA MUJER':'AMIGX',
        'PERSONA CERCANA HOMBRE':'AMIGX',
        'AMIGA':'AMIGX',
        'AMIGO':'AMIGX',
        'CONOCIDA':'OTRX',
        'CONOCIDO':'OTRX',
        'RELACION LABORAL':'OTRX',
        'OTRA':'OTRX',
    },
    'religion':{
        'NINGUNA':'NINGUNA',
        'CATOLICA':'SI',
        'CRISTIANA':'SI',
        'OTRA':'SI',
        'MORMONA':'SI',
        'TESTIGA DE JEHOVA':'SI',
        'BUDISTA':'SI',
        'JUDIA':'SI',
        'ANGLICANA':'SI',
        'MUSULMANA':'SI',
    },
}

#### Auxiliar para rangos de variables numéricas

In [ ]:
from pandas import qcut
from IPython.display import display

aux_dict = {}
for col in clean_dict['vars_num']:
    new_col = df[col].map(ile.clean_number).astype(float)
    orig_bins = qcut(new_col, q=6, retbins=True, duplicates='drop')[-1]
    aux_dict[col] = list(orig_bins[:-1])
display(aux_dict)

### Crear variables

In [ ]:
# Diccionario para crear variables
vars_dict = {}

# Para calcular la diferencia entre edades
vars_dict['edad_1a_menst'] = 'menarca'
vars_dict['edad_vida_sex'] = 'fsexual'
vars_dict['edad_actual'] = 'edad'

# Para comparar los métodos anticonceptivos
vars_dict['metodo_antes'] = 'anticonceptivo'
vars_dict['metodo_despues'] = 'panticoncep'

## Transformar

In [ ]:
df, cluster_cols = ile.wrangling_ile(df, clean_dict, vars_dict, export_result=True)

In [ ]:
ile.cool_print(f'\nLas {len(cluster_cols)} columnas para clustering son:\n{cluster_cols}\n\n', sleep_time=0.01)
df.sample()

## Clustering

In [ ]:
df, clustering_pipeline = ile.clustering_ile(df, cluster_cols, n_clusters=N_CLUSTERS)
df.sample()

## Checkpoint

In [1]:
## QUITAR COMENTARIOS SI YA SE HIZO EL CLUSTERING

from pathlib import Path
from pandas import read_csv

BASE_DIR = '/Users/efraflores/Desktop/EF/Contests/Datathon_CDMX/data' # Mac
# BASE_DIR = r'D:\Users\efras\Documents\EF\Contests\Datathon_CDMX\data' # Windows
RESOURCE_ID = '932b56bf-c5ec-4815-9814-370d58754002' ##### API SÓLO TRAE INFO 2016
FILE_NAME = 'interrupcion_embarazo'

from mariachis.models import InterrupcionEmbarazo
ile = InterrupcionEmbarazo(BASE_DIR, FILE_NAME)

cluster_cols = sorted(['panticoncep', 'desc_derechohab', 'p_consent', 'anticonceptivo', 'resultado_ile', 'procile_simplificada', 'religion', 'nivel_edu', 'procile_completa', 'desc_servicio', 'motiles', 'autoref', 'edocivil_descripcion', 'ocupacion', 'parentesco', 's_complica', 'consejeria', 'cconsejo', 'tanalgesico', 'c_dolor', 'rango_edad', 'rango_menarca', 'rango_fsexual', 'rango_sememb', 'rango_nhijos', 'rango_gesta', 'rango_c_num', 'rango_p_semgest', 'rango_p_diasgesta', 'naborto', 'npartos', 'ncesarea', 'nile', 'fsexual_vs_menarca', 'edad_vs_menarca', 'edad_vs_fsexual', 'antes_vs_despues', 'antes_vs_despues_detalle'])

df = read_csv(Path(BASE_DIR).joinpath(f'{FILE_NAME}_cluster.csv'), low_memory=False)
df.sample()

,año,mes,fingreso,autoref,edocivil_descripcion,edad,desc_derechohab,nivel_edu,ocupacion,religion,...,rango_gesta,rango_c_num,rango_p_semgest,rango_p_diasgesta,rango_fsexual_vs_menarca,rango_edad_vs_menarca,rango_edad_vs_fsexual,antes_vs_despues,antes_vs_despues_detalle,cluster
70886,2018.0,FEBRERO,2018-02-12,DESCONOCIDO,SOLTERX,19.0,NINGUNO,PREPARATORIA,DESCONOCIDO,SI,...,01,01,07,DESCONOCIDO,05 a 06,01 a 08,01 a 04,DIFERENTE,"antes: NINGUNO, después: DESCONOCIDO",B


## Perfilamiento

### Recuento

In [2]:
add_cols = ['fingreso_yearquarter','entidad']

In [3]:
# from string import ascii_uppercase
# # Imprime el diccionario para llenar
# aux = df[add_cols+cluster_cols+['cluster']].copy()
# {x:{y:'' for y in ascii_uppercase[:len(set(df['cluster']))]} for x in aux.columns}

In [4]:
ile.profiles(df[add_cols+cluster_cols+['cluster']])

### Interpretación

In [1]:
cluster_caract = {
  'fingreso_yearquarter': {
    'A': 'Fuerte en 2016',
    'B': 'Fuerte en 2018',
    'C': '',
    'D': '',
    'E': '',
    'F': 'Fuerte en 2020',
    'G': 'Año DESCONOCIDO',
    'H': 'Fuerte en 2017',
    'I': '',
    'J': 'Fuerte en 2019',
  },
 'entidad': {
    'A': 'Muy alta frec capitalina y EdoMéx',
    'B': 'Alta frec foránea',
    'C': '',
    'D': '',
    'E': 'Muy baja frec capitalina',
    'F': '',
    'G': 'Muy baja frec mexiquense',
    'H': 'Muy alta frec mexiquense',
    'I': '',
    'J': 'Muy baja frec capitalina',
  },
 'antes_vs_despues': {
    'A': 'De quienes mantienen el método, hay más aquí',
    'B': 'De quienes cambian el método, hay más aquí',
    'C': '',
    'D': '',
    'E': '',
    'F': '',
    'G': '',
    'H': '',
    'I': '',
    'J': '',
  },
 'antes_vs_despues_detalle': {
    'A': 'Antes condón y después no especifican',
    'B': 'Antes ninguno y después implante',
    'C': 'No cambian el condón',
    'D': 'Antes ninguno y después no especifican',
    'E': 'Antes ninguno y después no especifican',
    'F': 'Antes ninguno y después no especifican',
    'G': 'Antes ninguno y después no especifican',
    'H': 'Antes ninguno y después implante',
    'I': 'Antes condón y después no especifican',
    'J': 'Antes ninguno y después condón',
  },
 'anticonceptivo': {
    'A': '',
    'B': '',
    'C': '',
    'D': '',
    'E': '',
    'F': '',
    'G': '',
    'H': '',
    'I': '',
    'J': '',
  },
 'autoref': {
    'A': '',
    'B': '',
    'C': '',
    'D': '',
    'E': '',
    'F': '',
    'G': 'Acude directamente',
    'H': '',
    'I': 'Referida de otra unidad',
    'J': '',
  },
 'c_dolor': {
    'A': '',
    'B': '',
    'C': 'Dolor después del proced',
    'D': '',
    'E': 'Sin dolor después del proced',
    'F': 'Sin dolor después del proced',
    'G': 'Sin dolor después del proced',
    'H': '',
    'I': 'Dolor después del proced',
    'J': 'Sin dolor después del proced',
  },
 'cconsejo': {
    'A': '',
    'B': '',
    'C': 'Con consejo',
    'D': '',
    'E': '',
    'F': 'Con consejo',
    'G': 'Con consejo',
    'H': '',
    'I': 'Con consejo',
    'J': 'Con consejo',
  },
 'consejeria': {
    'A': 'De quienes no recibieron consejería, la mayoría está aquí',
    'B': 'Recibieron consejería',
    'C': '',
    'D': '',
    'E': '',
    'F': 'Recibieron consejería',
    'G': '',
    'H': 'Recibieron consejería',
    'I': '',
    'J': '',
  },
 'desc_derechohab': {
    'A': 'De quienes tienen seguro, la mayoría está aquí',
    'B': '',
    'C': '',
    'D': '',
    'E': '',
    'F': '',
    'G': 'No se sabe si tienen seguro',
    'H': 'De quienes NO tienen seguro, la mayoría está aquí',
    'I': 'No se sabe si tienen seguro',
    'J': '',
  },
 'desc_servicio': {
    'A': 'Servicio ILE',
    'B': 'Servicio ILE',
    'C': 'De quienes asistieron al servicio de cirugía ambulatoria, la mayoría está aquí',
    'D': '',
    'E': '',
    'F': '',
    'G': 'Atendidxs por especialidad de ginecología',
    'H': 'Servicio ILE',
    'I': 'Servicios variados en donde se otorga el proced',
    'J': '',
  },
 'edad_vs_fsexual': {
    'A': 'No hay información de su IVSA',
    'B': 'La ILE ocurre a los 4 años después de su IVSA',
    'C': 'La ILE ocurre a los 7 años después de su IVSA',
    'D': 'La ILE ocurre a los 8 años después de su IVSA',
    'E': 'La ILE ocurre a los 8 años después de su IVSA',
    'F': 'La ILE ocurre a los 6 años después de su IVSA',
    'G': 'La ILE ocurre a los 3 años después de su IVSA',
    'H': 'La ILE ocurre a los 4 años después de su IVSA',
    'I': 'La ILE ocurre más de 10 años después de su IVSA',
    'J': 'Muy probablemente la ILE ocurre en su primer encuentro sexual',
  },
 'edad_vs_menarca': {
    'A': '',
    'B': '',
    'C': '',
    'D': '',
    'E': '',
    'F': '',
    'G': '',
    'H': '',
    'I': '',
    'J': '',
  },
 'edocivil_descripcion': {
    'A': 'De quienes están casadxs, la mayoría está aquí',
    'B': 'De quienes están solterxs, la mayoría está aquí',
    'C': 'Unión libre',
    'D': '',
    'E': '',
    'F': 'De quienes están separadxs, la mayoría está aquí',
    'G': '',
    'H': '',
    'I': 'Unión libre',
    'J': '',
  },
 'fsexual_vs_menarca': {
    'A': 'No hay información de su IVSA o de su menarca',
    'B': 'IVSA a los 3 años post-menarca',
    'C': 'IVSA a los 6 años post-menarca',
    'D': 'IVSA a los 4 años post-menarca',
    'E': 'IVSA a los 5 años post-menarca',
    'F': 'IVSA a los 3 años post-menarca',
    'G': 'IVSA a los 3 años post-menarca',
    'H': 'IVSA a los 3 años post-menarca',
    'I': 'IVSA a los 3 años post-menarca',
    'J': 'IVSA a los 5 años post-menarca',
  },
 'motiles': {
    'A': '',
    'B': '',
    'C': '',
    'D': '',
    'E': '',
    'F': '',
    'G': '',
    'H': '',
    'I': '',
    'J': '',
  },
 'naborto': {
    'A': 'Se desconoce si ha tenido aborto',
    'B': 'De quienes no han tenido abortos, la mayoría está aquí',
    'C': '',
    'D': '',
    'E': '',
    'F': '',
    'G': '',
    'H': 'De quienes no han tenido abortos, la mayoría está aquí',
    'I': 'De quienes han tenido aborto(s), la mayoría está aquí',
    'J': '',
  },
 'ncesarea': {
    'A': 'Se desconoce si ha tenido cesárea',
    'B': 'De quienes no han tenido cesáreas, la mayoría está',
    'C': '',
    'D': '',
    'E': '',
    'F': '',
    'G': '',
    'H': '',
    'I': 'De quienes han tenido cesárea(s), la mayoría está aquí',
    'J': '',
  },
 'nile': {
    'A': 'Se desconoce si ha tenido interrupciones leagales del embarazo',
    'B': 'De quienes no han tenido ILE, la mayoría está',
    'C': '',
    'D': '',
    'E': '',
    'F': 'De quienes han tenido ILE, la mayoría está aquí',
    'G': '',
    'H': '',
    'I': '',
    'J': '',
  },
 'nivel_edu': {
    'A': 'De quienes tienen máximo primaria o sin acceso a la educación, la mayoría está aquí',
    'B': 'De quienes tienen máximo preparatoria o licenciatura, la mayoría está aquí',
    'C': 'Preparatoria',
    'D': 'Secundaria',
    'E': 'Secundaria',
    'F': 'Varios niveles educativos, pocas personas sin acceso',
    'G': 'De quienes se desconoce el nivel educativo, la mayoría está aquí',
    'H': 'De quienes tienen máximo licenciatura o posgrado, la mayoría está aquí',
    'I': 'Secundaria',
    'J': 'Preparatoria',
  },
 'npartos': {
    'A': 'O no se conoce o ha tenido un parto o más',
    'B': 'No ha tenido partos',
    'C': 'Sí ha tenido partos',
    'D': 'No ha tenido partos',
    'E': 'Sí ha tenido partos',
    'F': 'No ha tenido partos',
    'G': 'No ha tenido partos',
    'H': 'No ha tenido partos',
    'I': 'Mixto en haber tenido o no partos',
    'J': 'Sí ha tenido partos',
  },
 'ocupacion': {
    'A': 'Empleadx',
    'B': 'Se desconoce ocupación o es estudiante',
    'C': 'Trabajadorx del hogar no remuneradx',
    'D': 'Trabajadorx del hogar no remuneradx',
    'E': 'Trabajadorx del hogar no remuneradx',
    'F': 'De quienes están desempleadxs, la mayoría está aquí',
    'G': 'Estudiante y empleadx',
    'H': 'Empleadx',
    'I': 'Trabajadorx del hogar no remuneradx',
    'J': 'Estudiante',
  },
 'p_consent': {
    'A': 'De quienes no firmaron o no se sabe si se requisitó y firmó el consentimiento informado',
    'B': 'No se sabe si se requisitó y firmó el consentimiento informado',
    'C': '',
    'D': '',
    'E': '',
    'F': '',
    'G': '',
    'H': '',
    'I': 'Se requisitó y firmó el consentimiento informado',
    'J': '',
  },
 'panticoncep': {
    'A': '',
    'B': '',
    'C': '',
    'D': '',
    'E': '',
    'F': '',
    'G': '',
    'H': '',
    'I': '',
    'J': '',
  },
 'parentesco': {
    'A': '',
    'B': '',
    'C': '',
    'D': '',
    'E': '',
    'F': '',
    'G': 'Llegan acompañadxs',
    'H': '',
    'I': 'Pareja',
    'J': '',
  },
 'procile_completa': {
    'A': 'Terapia dual',
    'B': 'Terapia dual',
    'C': '',
    'D': 'Terapia dual',
    'E': 'Terapia dual',
    'F': '',
    'G': '',
    'H': 'Terapia dual',
    'I': 'Sólo requieren inducción de maduración cervical',
    'J': '',
  },
 'procile_simplificada': {
    'A': 'De quienes usaron medicamento en su procedimiento, la mayoría está aquí',
    'B': '',
    'C': 'De quienes tuvieron procedimiento AMEU, la mayoría está aquí',
    'D': '',
    'E': '',
    'F': '',
    'G': '',
    'H': 'De quienes usaron medicamento en su procedimiento, la mayoría está aquí',
    'I': '',
    'J': '',
  },
 'rango_c_num': {
    'A': 'Una cita, dos citas o no se sabe',
    'B': 'Dos citas previas',
    'C': 'Sin citas previas o 2 citas previas',
    'D': 'Una cita previa',
    'E': 'Una cita previa',
    'F': 'No se sabe cuántas citas previas',
    'G': 'A lo más una cita previa',
    'H': 'No se sabe cuántas citas previas',
    'I': 'Tres o más citas previas',
    'J': 'Dos citas previas',
  },
 'rango_edad': {
    'A': '19-35 años mayor frec 26-29',
    'B': '19-25 años mayor frec 19-21',
    'C': '22-29 años',
    'D': '22-25 años',
    'E': '22-35 años frec 22-25',
    'F': '19-35 años mayor frec 22-25',
    'G': '19-29 años',
    'H': '19-25 años mayor frec 22-25',
    'I': '22-35 años mayor frec 30-35',
    'J': '19-21 años',
  },
 'rango_fsexual': {
    'A': 'De quienes tuvieron su IVSA menor a 8 años, la mayoría está aquí',
    'B': 'IVSA 08-15 años',
    'C': '',
    'D': '',
    'E': '',
    'F': '',
    'G': '',
    'H': 'IVSA 08-11 y >= 19',
    'I': '',
    'J': '',
  },
 'rango_gesta': {
    'A': '2 o más gesta',
    'B': '1 gesta',
    'C': '2 gesta',
    'D': '1 gesta',
    'E': '3 gesta',
    'F': '1-3 gesta',
    'G': '1-2 gesta',
    'H': '1-2 gesta',
    'I': '2-3 gesta',
    'J': '1 gesta',
  },
 'rango_menarca': {
    'A': 'menarca a los 12 años',
    'B': 'menarca de 8-12 años',
    'C': 'menarca de 8-11 años',
    'D': 'menarca a los 13 años',
    'E': 'menarca de 8-11 años',
    'F': 'menarca de 12 años',
    'G': 'menarca de 12 años',
    'H': 'menarca de 8-14 años',
    'I': 'menarca de 12 años',
    'J': 'menarca de 12 años',
  },
 'rango_nhijos': {
    'A': '1 hijx',
    'B': 'Sin hijxs',
    'C': '1 hijx',
    'D': 'Sin hijxs',
    'E': '2 hijxs',
    'F': 'O no tiene o se desconoce número de hijxs',
    'G': 'Sin hijxs',
    'H': 'Sin hijxs',
    'I': '1 o más hijxs',
    'J': 'Sin hijxs',
  },
 'rango_p_diasgesta': {
    'A': '',
    'B': '',
    'C': '',
    'D': '',
    'E': '',
    'F': '',
    'G': '',
    'H': '',
    'I': '',
    'J': '',
  },
 'rango_p_semgest': {
    'A': '7 semanas de gesta',
    'B': '',
    'C': '10 semanas de gesta o más',
    'D': '',
    'E': '7 semanas de gesta',
    'F': '',
    'G': '6-8 semanas de gesta',
    'H': '',
    'I': '6-8 semanas de gesta',
    'J': '7 semanas de gesta',
  },
 'rango_sememb': {
    'A': '',
    'B': '',
    'C': '',
    'D': '',
    'E': '',
    'F': '',
    'G': '',
    'H': '',
    'I': '',
    'J': '',
  },
 'religion': {
    'A': '',
    'B': '',
    'C': '',
    'D': '',
    'E': '',
    'F': 'De quienes respondieron no tener religión, la mayoría está aquí',
    'G': '',
    'H': '',
    'I': '',
    'J': 'De quienes no se sabe si tienen religión, la mayoría está aquí',
  },
 'resultado_ile': {
    'A': 'De quienes respondieron "Otro" el procedimiento ILE, la mayoría está aquí',
    'B': 'De quienes completaron el procedimiento ILE, la mayoría está aquí',
    'C': '',
    'D': '',
    'E': '',
    'F': 'De quienes no se sabe completitud del procedimiento ILE, la mayoría está aquí',
    'G': '',
    'H': '',
    'I': 'De quienes no se sabe completitud del procedimiento ILE, la mayoría está aquí',
    'J': '',
  },
 's_complica': {
    'A': 'Se desconoce si hubo complicaciones en el procedimiento',
    'B': 'Se desconoce si hubo complicaciones en el procedimiento',
    'C': '',
    'D': 'Se desconoce si hubo complicaciones en el procedimiento',
    'E': 'Se desconoce si hubo complicaciones en el procedimiento',
    'F': 'De los procedimientos que no se complican, la mayoría está aquí',
    'G': '',
    'H': 'De los procedimientos que no se complican, la mayoría está aquí',
    'I': '',
    'J': 'De los procedimientos que se complican, la mayoría está aquí',
  },
 'tanalgesico': {
    'A': '',
    'B': 'Se prescribe analgésico',
    'C': 'Se prescribe analgésico',
    'D': '',
    'E': 'No se prescribe analgésico',
    'F': 'No se prescribe analgésico',
    'G': 'No se prescribe analgésico',
    'H': '',
    'I': 'Se prescribe analgésico',
    'J': 'No se prescribe analgésico',
  },
 'cluster': {
    'A': '13.0%',
    'B': '13.4%',
    'C': '08.7%',
    'D': '10.5%',
    'E': '06.1%',
    'F': '12.1%',
    'G': '08.1%',
    'H': '13.1%',
    'I': '08.5%',
    'J': '06.5%',
}}

### Resumen

In [8]:
from pandas import DataFrame

DataFrame(cluster_caract)#.T.to_dict()

,fingreso_yearquarter,entidad,antes_vs_despues,antes_vs_despues_detalle,anticonceptivo,autoref,c_dolor,cconsejo,consejeria,desc_derechohab,...,rango_menarca,rango_nhijos,rango_p_diasgesta,rango_p_semgest,rango_sememb,religion,resultado_ile,s_complica,tanalgesico,cluster
A,Fuerte en 2016,Muy alta frec capitalina y EdoMéx,"De quienes mantienen el método, hay más aquí",Antes condón y después no especifican,,,,,"De quienes no recibieron consejería, la mayorí...","De quientes tienen seguro, la mayoría está aquí",...,menarca a los 12 años,1 hijx,,7 semanas de gesta,,,"De quienes respondieron ""Otro"" el procedimient...",Se desconoce si hubo complicaciones en el proc...,,13.0%
B,Fuerte en 2018,Alta frec foránea,"De quienes cambian el método, hay más aquí",Antes ninguno y después implante,,,,,Recibieron consejería,,...,menarca de 8-12 años,Sin hijxs,,,,,"De quienes completaron el procedimiento ILE, l...",Se desconoce si hubo complicaciones en el proc...,Se prescribe analgésico,13.4%
C,,,,No cambian el condón,,,Dolor después del proced,Con consejo,,,...,menarca de 8-11 años,1 hijx,,10 semanas de gesta o más,,,,,Se prescribe analgésico,08.7%
D,,,,Antes ninguno y después no especifican,,,,,,,...,menarca a los 13 años,Sin hijxs,,,,,,Se desconoce si hubo complicaciones en el proc...,,10.5%
E,,Muy baja frec capitalina,,Antes ninguno y después no especifican,,,Sin dolor después del proced,,,,...,menarca de 8-11 años,2 hijxs,,7 semanas de gesta,,,,Se desconoce si hubo complicaciones en el proc...,No se prescribe analgésico,06.1%
F,Fuerte en 2020,,,Antes ninguno y después no especifican,,,Sin dolor después del proced,Con consejo,Recibieron consejería,,...,menarca de 12 años,O no tiene o se desconoce número de hijxs,,,,"De quienes respondieron no tener religión, la ...",De quienes no se sabe completitud del procedim...,"De los procedimientos que no se complican, la ...",No se prescribe analgésico,12.1%
G,Año DESCONOCIDO,Muy baja frec mexiquense,,Antes ninguno y después no especifican,,Acude directamente,Sin dolor después del proced,Con consejo,,No se sabe si tienen seguro,...,menarca de 12 años,Sin hijxs,,6-8 semanas de gesta,,,,,No se prescribe analgésico,08.1%
H,Fuerte en 2017,Muy alta frec mexiquense,,Antes ninguno y después implante,,,,,Recibieron consejería,"De quientes NO tienen seguro, la mayoría está ...",...,menarca de 8-14 años,Sin hijxs,,,,,,"De los procedimientos que no se complican, la ...",,13.1%
I,,,,Antes condón y después no especifican,,Referida de otra unidad,Dolor después del proced,Con consejo,,No se sabe si tienen seguro,...,menarca de 12 años,1 o más hijxs,,6-8 semanas de gesta,,,De quienes no se sabe completitud del procedim...,,Se prescribe analgésico,08.5%
J,Fuerte en 2019,Muy baja frec capitalina,,Antes ninguno y después condón,,,Sin dolor después del proced,Con consejo,,,...,menarca de 12 años,Sin hijxs,,7 semanas de gesta,,"De quienes no se sabe si tienen religión, la m...",,"De los procedimientos que se complican, la may...",No se prescribe analgésico,06.5%
